In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm,pyplot
from matplotlib.colors import LinearSegmentedColormap
import os

In [2]:
colors1 = plt.cm.terrain(np.linspace(0., 0.75, 128))
colors2 = plt.cm.terrain(np.linspace(0.75, 1, 128))
colors = np.vstack((colors1, colors2))
mymap = LinearSegmentedColormap.from_list('rixs', colors)

In [3]:
def lorentz3D(x, y, amp, xc, yc, wid):
    return amp*(wid/((x-xc)**2+(y-yc)**2+wid**2)**1.5)

# We can broaden in x axis and then broaden in y axis instead of 3D broadening
def lorentz(x, amp, cen, wid):
    return (amp)*(wid/((x-cen)**2 + wid**2))

In [26]:
def read_file(flist,xdata,ydata,b_ab,b_em,eloss=False):
    x,y = np.meshgrid(xdata,ydata)
    z_ab = np.zeros((xdata.size,xdata.size))
    z = np.zeros((xdata.size,xdata.size))
    for fname in flist:
        with open(fname) as rixs:
            lines = rixs.readlines()[1:]
            for l in lines:
                ab = float(l.split()[0])
                el = float(l.split()[1])
                elind = int((el-elmin)/(elmax-elmin)*nedos)
                peak = float(l.split()[2])
                if (peak < 1e-6): continue
#                 print(ab,el,peak)
                # Broaden in absorption
                z_ab[:,elind] += lorentz(y_ax,peak,ab,b_ab)
    # Broaden in energy loss
    bd_list = []
    for yy in range(0,nedos):
        if(np.any(z_ab[:,yy])): 
            bd_list.append(yy)

    for bd_ind in bd_list:
        for xx in range(0,nedos):
            if (z_ab[xx][bd_ind] != 0):
                z[xx,:] += lorentz(x_ax,z_ab[xx][bd_ind],x_ax[bd_ind],b_em)

    # Wipe out anything with energy loss < 0
    if (eloss):
        zeroind = int((-elmin)/(elmax-elmin)*nedos)
        for yy in range(0,zeroind):
            z[:,yy].fill(0)
    else:
        for x_ind in range(0,nedos):
            for y_ind in range(0,nedos):
                if x_ax[x_ind] >= y_ax[y_ind]:
                    z[y_ind,x_ind:].fill(0)
                    continue;
    return z

def read_dir_xas(dir_name,xdata,ydata,b_ab=0.3,b_em=0.3,edge = "L",pol_in = "XYZ",pol_out = "XYZ",eloss=False):
    zdata = np.zeros((xdata.size,xdata.size))
    flist = []
    for pin in pol_in:
        for pout in pol_out:  
            filename = dir_name+"/RIXS_"+edge+"edge_"+pin+"_"+pout+".txt"
            flist.append(filename)
    zdata += read_file(flist,xdata,ydata,b_ab,b_em,eloss)
    return zdata

In [33]:
nedos = 2000
abmin = -25
abmax = 25
elmin = -30
elmax = 30
eloss = False
x_ax = np.linspace(elmin,elmax,nedos)
y_ax = np.linspace(abmin,abmax,nedos)
x,y = np.meshgrid(x_ax,y_ax)

# Another Output format
nedos = 1000
x = np.linspace(0,3,nedos)
y = np.linspace(-15,15,nedos)
x,y = np.meshgrid(x,y)
z = np.zeros((nedos,nedos))
z = np.loadtxt("rixs_mat.txt", delimiter=',')

In [1]:
shift = 0
edge = "L3"
# z = read_dir_xas("./WORK_DIR/LiCoO2/d6L/final",x_ax,y_ax,0.3,0.3,edge[0])
z = read_dir_xas("./WORK_DIR/LiNiO2/v1/d8L2",x_ax,y_ax,0.3,0.3,edge[0],"X","XYZ",eloss)
if (eloss): c = "gnuplot"
else: c = mymap
    
plt.pcolormesh(x+shift, y+shift, z.reshape(x.shape),shading="auto",cmap=c)
# plt.title("d8 K edge NCT RIXS x/y, dppsigma = 0.25")
if (eloss): plt.xlabel("Loss Energy (eV)")
else: plt.xlabel("Emission Energy (eV)")
plt.ylabel("Incident Energy (eV)")
# plt.clim([0,0.2])
if (edge == 'L'):
    plt.ylim([-10+shift,25+shift])
if (edge == 'K'):
    plt.ylim([-20+shift,-2+shift])  
    plt.ylim([-10+shift,5+shift])  
if (not eloss):
    plt.xlim([plt.axis()[2]-10+shift,plt.axis()[3]-5+shift])
else:
    plt.xlim([0+shift,7+shift])

if (edge == 'L3'):
    plt.ylim([-5+shift,4+shift]) 
    if (not eloss): plt.xlim([-15+shift,5+shift])
    else: plt.xlim([0+shift,15+shift])
plt.rcParams["figure.figsize"] = (6,6)
plt.rc('font', size=13)   
if (not eloss):
    lims = np.linspace(-50+shift,50+shift,nedos)
    plt.plot(lims,lims,linewidth=1,linestyle='--',color="yellow")
plt.rcParams["figure.figsize"] = (7,6)
plt.colorbar()
plt.title("d8L "+edge+" edge RIXS (LiNiO2), incoming polarization: X")
plt.savefig('/Users/seanhsu/Desktop/School/Research/Program File/ED/plot.png')
plt.show()

NameError: name 'read_dir_xas' is not defined